In [1]:
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers.legacy import SGD as SGD_legacy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import argmax
import tensorflow as tf
from sklearn.metrics import accuracy_score
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Definir las rutas para los datos de entrenamiento y validación
train_data_dir = '/kaggle/input/100-bird-species/train'
validation_data_dir = '/kaggle/input/100-bird-species/valid'

In [3]:
# Tamaño de la imagen y tamaño del lote
img_width, img_height = 28, 28
batch_size = 64

In [4]:
# Crear un ImageDataGenerator para los datos de entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,         # normalizar valores de píxeles a [0,1]
    shear_range=0.2,         # transformaciones de cizallamiento
    zoom_range=0.2,          # transformaciones de zoom
    horizontal_flip=True     # volteos horizontales
)

In [5]:
# Crear un ImageDataGenerator para los datos de validación (solo reescalado)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Cargar y preparar los datos de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 84635 images belonging to 525 classes.


In [7]:
# Cargar y preparar los datos de validación
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Obtener etiquetas verdaderas del generador de validación
true_labels = validation_generator.classes

Found 2625 images belonging to 525 classes.


In [8]:
num_classes = len(os.listdir(train_data_dir))

In [9]:
def define_cnn_model(input_shape, num_classes):
    """
    Definir un modelo de Red Neuronal Convolucional (CNN).

    Parámetros:
    - input_shape (tuple): La forma de los datos de entrada (por ejemplo, (img_width, img_height, num_channels)).
    - num_classes (int): El número de clases en la tarea de clasificación.

    Devuelve:
    - model (Sequential): El modelo CNN definido.
    """
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(500, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(num_classes, activation='softmax'))
    
    opt = Adam(learning_rate=0.001)
    #opt = SGD_legacy(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
# Entrenar el modelo CNN utilizando ImageDataGenerator
model = define_cnn_model((img_width, img_height, 3), num_classes)   # Suponiendo imágenes RGB
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
num_train_samples = len(train_generator.filenames)
num_validation_samples = len(validation_generator.filenames)

In [11]:
# Establecer la ejecución ansiosa
tf.config.run_functions_eagerly(True)

In [12]:
# Ajustar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    #callbacks=[early_stopping]
)

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1322/1322 [==============================] - 224s 168ms/step - loss: 4.6369 - accuracy: 0.1193 - val_loss: 3.8855 - val_accuracy: 0.2024
Epoch 2/20
1322/1322 [==============================] - 212s 160ms/step - loss: 3.3381 - accuracy: 0.2855 - val_loss: 2.8712 - val_accuracy: 0.3617
Epoch 3/20
1322/1322 [==============================] - 207s 157ms/step - loss: 2.8022 - accuracy: 0.3749 - val_loss: 2.5776 - val_accuracy: 0.4242
Epoch 4/20
1322/1322 [==============================] - 207s 156ms/step - loss: 2.4653 - accuracy: 0.4333 - val_loss: 2.4636 - val_accuracy: 0.4486
Epoch 5/20
1322/1322 [==============================] - 206s 156ms/step - loss: 2.2075 - accuracy: 0.4829 - val_loss: 2.4486 - val_accuracy: 0.4638
Epoch 6/20
1322/1322 [==============================] - 206s 156ms/step - loss: 1.9979 - accuracy: 0.5204 - val_loss: 2.2008 - val_accuracy: 0.5053
Epoch 7/20
1322/1322 [==============================] - 207s 157ms/step - loss: 1.8424 - accuracy: 0.5510 - val_loss: 2.143

In [13]:
# Evaluar el modelo en el conjunto de prueba
test_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 2625 images belonging to 525 classes.


In [14]:
# Predecir las etiquetas para el conjunto de prueba
predictions = model.predict_generator(test_generator, steps=num_validation_samples // batch_size + 1)
predicted_labels = [argmax(pred) for pred in predictions]

/tmp/ipykernel_1540/4100624326.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_generator, steps=num_validation_samples // batch_size + 1)


In [15]:
# Asegurar que las longitudes sean iguales
true_labels = true_labels[:len(predicted_labels)]

In [16]:
# Evaluar el modelo en el conjunto de prueba
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 57.18%


In [17]:
from keras.models import load_model
model.save('my_model.keras')

In [20]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

# Load the trained model
model = load_model('my_model.keras')

# Load and prepare an image
def prepare_image(file_path, target_size):
    img = load_img(file_path, target_size=target_size)  # Load the image
    img = img_to_array(img)  # Convert the image to a numpy array
    img = np.expand_dims(img, axis=0)  # Add a batch dimension
    img = img / 255.0  # Scale pixel values to [0, 1]
    return img

image_path = '/kaggle/input/100-bird-species/test/AMERICAN COOT/3.jpg'
prepared_img = prepare_image(image_path, target_size=(28, 28))  # Ensure target_size is same as model's input size

# Predict the class
prediction = model.predict(prepared_img)
predicted_class = np.argmax(prediction, axis=1)

# Output the predicted class
print(f"Predicted class: {predicted_class[0]}")

1/1 [==============================] - 0s 25ms/step
Predicted class: 16
